# NLP u ugostiteljskoj industriji

In [ ]:
import pandas as pd
import re

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
df = pd.read_csv('https://drive.google.com/uc?export=download&id=1Eb4KOL-0jIr-DXu0eqaIzKQWrO8d0Vzo')
df.head()

,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4* experience hotel monaco seat...,3
3,"unique, great stay, wonderful time hotel monac...",5
4,"great stay great stay, went seahawk game aweso...",5


In [ ]:
# Pretprocesiranje teksta
df['Review'] = df['Review'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x).lower())

In [ ]:
# Kategoriziranje ocjena
def label_rating(rating):
    if rating <= 2:
        return 'Negative'
    elif rating == 3:
        return 'Neutral'
    else:
        return 'Positive'

df['Label'] = df['Rating'].apply(label_rating)

In [ ]:
# Tokenizacija
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(df['Review'])
sequences = tokenizer.texts_to_sequences(df['Review'])

In [ ]:
# Padding
max_len = max([len(x) for x in sequences])
X = pad_sequences(sequences, maxlen=max_len)

In [ ]:
# One-hot kodiranje oznaka
labels = pd.get_dummies(df['Label']).values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

# Definiranje modela
model = Sequential()
model.add(Embedding(5000, 128, input_length=max_len)) # 5000 = veličina vokabulara, 128 = dimenzija embeddinga
model.add(SimpleRNN(64)) # broj jedinica u RNN sloju
model.add(Dense(3, activation='softmax')) # 3 izlazne jedinice za 3 klase

# Kompilacija modela
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Treniranje modela
history = model.fit(X_train, y_train, batch_size=128, epochs=10, validation_data=(X_val, y_val))

Epoch 1/10
97/97 [==============================] - 184s 2s/step - loss: 0.7609 - accuracy: 0.7265 - val_loss: 0.7078 - val_accuracy: 0.7389
Epoch 2/10
97/97 [==============================] - 182s 2s/step - loss: 0.5202 - accuracy: 0.8034 - val_loss: 0.5434 - val_accuracy: 0.8050
Epoch 3/10
97/97 [==============================] - 183s 2s/step - loss: 0.2634 - accuracy: 0.9062 - val_loss: 0.5425 - val_accuracy: 0.8187
Epoch 4/10
97/97 [==============================] - 176s 2s/step - loss: 0.0933 - accuracy: 0.9784 - val_loss: 0.6258 - val_accuracy: 0.7997
Epoch 5/10
97/97 [==============================] - 184s 2s/step - loss: 0.0236 - accuracy: 0.9977 - val_loss: 0.6873 - val_accuracy: 0.8119
Epoch 6/10
97/97 [==============================] - 186s 2s/step - loss: 0.0074 - accuracy: 0.9997 - val_loss: 0.7324 - val_accuracy: 0.8155
Epoch 7/10
97/97 [==============================] - 186s 2s/step - loss: 0.0038 - accuracy: 1.0000 - val_loss: 0.7734 - val_accuracy: 0.8121
Epoch 8/10
97

In [ ]:
model.evaluate(X_test, y_test)

129/129 [==============================] - 17s 131ms/step - loss: 0.8279 - accuracy: 0.8107


[0.8278778791427612, 0.8106855154037476]

In [ ]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping

model = Sequential()
model.add(Embedding(5000, 128, input_length=max_len))
model.add(SimpleRNN(64, return_sequences=True))
model.add(Dropout(0.5))
model.add(SimpleRNN(64))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(X_train, y_train, epochs=10, batch_size=128, validation_data=(X_val, y_val), callbacks=[early_stopping])

Epoch 1/10
97/97 [==============================] - 320s 3s/step - loss: 0.8388 - accuracy: 0.6894 - val_loss: 0.8159 - val_accuracy: 0.7416
Epoch 2/10
97/97 [==============================] - 295s 3s/step - loss: 0.6483 - accuracy: 0.7586 - val_loss: 1.1851 - val_accuracy: 0.7389
Epoch 3/10
97/97 [==============================] - 312s 3s/step - loss: 0.6878 - accuracy: 0.7525 - val_loss: 0.9183 - val_accuracy: 0.7418
Epoch 4/10
97/97 [==============================] - 315s 3s/step - loss: 0.6113 - accuracy: 0.7880 - val_loss: 0.8004 - val_accuracy: 0.7816
Epoch 5/10
97/97 [==============================] - 295s 3s/step - loss: 0.5284 - accuracy: 0.8193 - val_loss: 0.8236 - val_accuracy: 0.7877
Epoch 6/10
97/97 [==============================] - 292s 3s/step - loss: 0.4899 - accuracy: 0.8298 - val_loss: 0.6279 - val_accuracy: 0.8170
Epoch 7/10
97/97 [==============================] - 309s 3s/step - loss: 0.4712 - accuracy: 0.8371 - val_loss: 0.6386 - val_accuracy: 0.8114
Epoch 8/10
97

In [ ]:
model.evaluate(X_test, y_test)

129/129 [==============================] - 27s 206ms/step - loss: 0.5391 - accuracy: 0.8192


[0.5390658378601074, 0.8192241787910461]